In [200]:
import pandas as pd
import numpy as np
import sklearn as sk
from sklearn.model_selection import train_test_split
eps=np.finfo(float).eps
from binarytree import tree,Node
from sklearn.metrics import classification_report, confusion_matrix ,accuracy_score
from sklearn.tree import DecisionTreeClassifier
from operator import itemgetter
import copy
import math

In [201]:
df=pd.read_csv("train.csv")

In [202]:
X =df.drop(['left','sales','salary'],axis=1)
X = pd.concat([X,pd.get_dummies(df['sales'], prefix='sales')],axis=1)
X = pd.concat([X,pd.get_dummies(df['salary'], prefix='salary')],axis=1)
y=df['left']

In [203]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
orignalX=copy.deepcopy(X_train)
orignalY=copy.deepcopy(y_train)
testX=copy.deepcopy(X_test)
testY=copy.deepcopy(y_test)

orignalXEntropy=copy.deepcopy(X_train)
orignalYEntropy=copy.deepcopy(y_train)
testXEntropy=copy.deepcopy(X_test)
testYEntropy=copy.deepcopy(y_test)

orignalXGini=copy.deepcopy(X_train)
orignalYGini=copy.deepcopy(y_train)
testXGini=copy.deepcopy(X_test)
testYGini=copy.deepcopy(y_test)

orignalXCR=copy.deepcopy(X_train)
orignalYCR=copy.deepcopy(y_train)
testXCR=copy.deepcopy(X_test)
testYCR=copy.deepcopy(y_test)


In [204]:
def Gini(OrignalXGini,OrignalYGini,testXGini,testYGini):
    def findParentGini(df):
        if df.empty==True:
            return
        classCol=df.keys()[-1]
        #yeh sirf header return karega
        resultValues=df[classCol].unique()
        #all ouput values in output column
        gini=2
        for value in resultValues:
            fraction=df[classCol].value_counts()[value]/(len(df[classCol])+eps)
            gini*=fraction
        return abs(gini)
    def findRelativeGini(df,attribute):
        if df.empty==True:
            return
        classCol= df.keys()[-1] 
        #last col assumed as result 
        #getting target class 
        resultValues=df[classCol].unique() 
        attributeNames=df[attribute].unique()
        gini2 = 0
        for attr in attributeNames:
            gini = 2
            for value in resultValues:
                #temp&hot with yes
                num = len(df[attribute][df[attribute]==attr][df[classCol]==value])
                #total hot
                den = len(df[attribute][df[attribute]==attr])
                fraction = num/(den+eps)
                gini *= fraction
            #entropy attribute/total*(entropy attribute_values)
            fraction2 = den/len(df)  
            gini2 += fraction2*gini
        return abs(gini2)  
    def get_subtable(df, node,value):
        return df[df[node] == value].reset_index(drop=True)
    def get_subtableSmaller(df, node,value):
        if df.empty==True:
            return
        return df[df[node] < value].reset_index(drop=True)

    def get_subtableGreater(df, node,value):
        if df.empty==True:
            return
        return df[df[node] >= value].reset_index(drop=True)

    def bestAttributeGini(df):
        if df.empty==True:
            return
        infoGain=[]
        for key in df.keys()[:-1]:
            infoGain.append(findParentGini(df)-findRelativeGini(df,key))
        return df.keys()[:-1][np.argmax(infoGain)]



    df=pd.concat([orignalXGini,orignalYGini],axis=1)
    print(df)
    splitpoint={}
    numeric=['last_evaluation','satisfaction_level','average_montly_hours','time_spend_company','number_project']
    for col in numeric:
        df.sort_values(col,inplace=True)
        prev=None;
        value=0;
        max1=0;
#         print(col)
        for index,row in df.iterrows():
            if prev is not None:
                if prev['left'] != row['left']:
                    mid=(float(prev[col])+float(row[col]))/2
                    subtableLeft=get_subtableSmaller(df,col,mid)
                    subtableRight=get_subtableGreater(df,col,mid)
                    fraction=len(subtableLeft)/len(df)
                    fraction1=len(subtableRight)/len(df)
                    Gini1=findParentGini(df)
    #                 print(subtableLeft)
    #                 a=input()
                    firstSubtable=0
                    secondSubtable=0
                    if subtableLeft.empty:
                        firstSubtable=0
                    else:
                        firstSubtable=fraction*findParentGini(subtableLeft)
    #                 print(subtableRight)
                    if subtableRight.empty:
                        secondSubtable=0
                    else:
                        secondSubtable=fraction1*findParentGini(subtableRight)

                    Gini2=firstSubtable+secondSubtable
                    ig1=abs(Gini1)-abs(Gini2)

                    if ig1>max1:
                        max1=ig1
                        value=mid
            prev=row
        splitpoint[col]=value 
        for index,row1 in orignalXGini.iterrows():
            if float(row1[col])<value:
                orignalXGini.at[index,col]=0
            else:
                orignalXGini.at[index,col]=1


    # print(X_train)
    # print(value)
    # print(max1)
    df=pd.concat([orignalXGini,orignalYGini],axis=1)
    #df

    class node:
        def __init__(self,value,left=None,right=None):
            self.value=value
            self.left=left
            self.right=right
            self.positive=0
            self.negative=0

    def buildTreeGini(df):
        if len(df.columns)==1:
            return

        resultLabel= df.keys()[-1] 
        clValue,counts = np.unique(df[resultLabel],return_counts=True)
        attribute_name=bestAttributeGini(df)
    #     attValue = np.unique(df[attribute_name])
        root=node(attribute_name)

        if len(counts)<=1:
            if clValue[0]==0:
                root.negative=counts[0]
            else:
                root.positive=counts[0]
        else:
            root.positive=counts[1]
            root.negative=counts[0]


    #         for value in attValue:
    #             print(type(value))

            subtable0 = get_subtable(df,attribute_name,0)
            subtable0 =subtable0.drop([attribute_name],axis=1)
            subtable1 = get_subtable(df,attribute_name,1)
            subtable1 =subtable1.drop([attribute_name],axis=1)
            clValue,countleft = np.unique(subtable0[resultLabel],return_counts=True)
            clValue1,countright = np.unique(subtable1[resultLabel],return_counts=True)
            if(len(countleft)<=1):
                pass
            else:
                root.left=buildTreeGini(subtable0) #Calling the function recursively
            if(len(countright)<=1):
                pass
            else:
                root.right=buildTreeGini(subtable1)


        return root

    # print(df)
    root=buildTreeGini(df)
#     print(root.value)

#     print(root.right.value)
#     print(root.right.left)
#     print(root.left.value)
#     print(root.left.left.value)
#     print(root.left.right.value)
#     print(root.positive,root.negative)

    def testing(row1,root):
    #     if root==None:
    #         return 
        #print(root.value)
        if root.left==None and root.right==None:
            if root.positive > root.negative:
                y1.append(1)
            else:
                y1.append(0)
            return
    #     for i in row1.keys():
    #         if i==root.value:
        i=root.value
        if i in row1:
            
            if row1[i]==0:
        #         print("000")
                if root.left==None:
                    if root.positive > root.negative:
                        y1.append(1)
                        return
                    else:
                        y1.append(0)
                        return
                else:
                    testing(row1,root.left)
            else:
        #         print("111")
                if root.right==None:
                    if root.positive > root.negative:
                        y1.append(1)
                        return
                    else:
                        y1.append(0)
                        return 
                else:
                    testing(row1,root.right)
        else:
            if root.left==None:
                if root.positive > root.negative:
                    y1.append(1)
                    return
                else:
                    y1.append(0)
                    return
            else:
                testing(row1,root.left)
            
        



    #X_test
#     print(splitpoint)

    def testingData(df):
        for index,row in df.iterrows():
            testing(row,root)

    y1=[] 
    for col in numeric:
        value=splitpoint[col]
        for index,row1 in testXGini.iterrows():
                if float(row1[col])<value:
                    testXGini.at[index,col]=0
                else:
                    testXGini.at[index,col]=1
#     testXGini
    testingData(testXGini)  
#     print(len(y1))
    return y1

In [205]:
def Entropy(orignalXEntropy,orignalYEntropy,testXEntropy,testYEntropy):
    
    def findParentEntropy(df):
        if df.empty==True:
            return
        classCol=df.keys()[-1]
        #yeh sirf header return karega
        resultValues=df[classCol].unique()
        #all ouput values in output column
        entropy=0
        for value in resultValues:
            fraction=df[classCol].value_counts()[value]/(len(df[classCol])+eps)
            entropy+=-fraction*np.log2(fraction+eps)
        return abs(entropy)



    def findRelativeEntropy(df,attribute):
        if df.empty==True:
            return
        classCol= df.keys()[-1] 
        #last col assumed as result 
        #getting target class 
        resultValues=df[classCol].unique() 
        attributeNames=df[attribute].unique()
        entropy2 = 0
        for attr in attributeNames:
            entropy = 0
            for value in resultValues:
                #temp&hot with yes
                num = len(df[attribute][df[attribute]==attr][df[classCol]==value])
                #total hot
                den = len(df[attribute][df[attribute]==attr])
                fraction = num/(den+eps)
                entropy += -fraction*np.log2(fraction+eps)
            #entropy attribute/total*(entropy attribute_values)
            fraction2 = den/len(df)  
            entropy2 += -fraction2*entropy 
        return abs(entropy2)

    def get_subtable(df, node,value):
        return df[df[node] == value].reset_index(drop=True)


    def get_subtableSmaller(df, node,value):
        if df.empty==True:
            return
        return df[df[node] < value].reset_index(drop=True)

    def get_subtableGreater(df, node,value):
        if df.empty==True:
            return
        return df[df[node] >= value].reset_index(drop=True)

    def bestAttribute(df):
        if df.empty==True:
            return
        infoGain=[]
        for key in df.keys()[:-1]:
            infoGain.append(findParentEntropy(df)-findRelativeEntropy(df,key))
        return df.keys()[:-1][np.argmax(infoGain)]



    df=pd.concat([orignalXEntropy,orignalYEntropy],axis=1)
    print(df)
    splitpoint={}
    numeric=['last_evaluation','satisfaction_level','average_montly_hours','time_spend_company','number_project']
    for col in numeric:
        df.sort_values(col,inplace=True)
        prev=None;
        value=0;
        max1=0;
#         print(col)
        for index,row in df.iterrows():
            if prev is not None:
                if prev['left'] != row['left']:
                    mid=(float(prev[col])+float(row[col]))/2
                    subtableLeft=get_subtableSmaller(df,col,mid)
                    subtableRight=get_subtableGreater(df,col,mid)
                    fraction=len(subtableLeft)/len(df)
                    fraction1=len(subtableRight)/len(df)
                    entropy1=findParentEntropy(df)
    #                 print(subtableLeft)
    #                 a=input()
                    firstSubtable=0
                    secondSubtable=0
                    if subtableLeft.empty:
                        firstSubtable=0
                    else:
                        firstSubtable=fraction*findParentEntropy(subtableLeft)
    #                 print(subtableRight)
                    if subtableRight.empty:
                        secondSubtable=0
                    else:
                        secondSubtable=fraction1*findParentEntropy(subtableRight)

                    entropy2=firstSubtable+secondSubtable
                    ig1=abs(entropy1)-abs(entropy2)

                    if ig1>max1:
                        max1=ig1
                        value=mid
            prev=row
        splitpoint[col]=value 
        for index,row1 in orignalXEntropy.iterrows():
            if float(row1[col])<value:
                orignalXEntropy.at[index,col]=0
            else:
                orignalXEntropy.at[index,col]=1


    # print(X_train)
    # print(value)
    # print(max1)
    df=pd.concat([orignalXEntropy,orignalYEntropy],axis=1)
    #df

    class node:
        def __init__(self,value,left=None,right=None):
            self.value=value
            self.left=left
            self.right=right
            self.positive=0
            self.negative=0

    def buildTree(df):
        if len(df.columns)==1:
            return

        resultLabel= df.keys()[-1] 
        clValue,counts = np.unique(df[resultLabel],return_counts=True)
        attribute_name=bestAttribute(df)
    #     attValue = np.unique(df[attribute_name])
        root=node(attribute_name)

        if len(counts)<=1:
            if clValue[0]==0:
                root.negative=counts[0]
            else:
                root.positive=counts[0]
        else:
            root.positive=counts[1]
            root.negative=counts[0]


    #         for value in attValue:
    #             print(type(value))

            subtable0 = get_subtable(df,attribute_name,0)
            subtable0 =subtable0.drop([attribute_name],axis=1)
            subtable1 = get_subtable(df,attribute_name,1)
            subtable1 =subtable1.drop([attribute_name],axis=1)
            clValue,countleft = np.unique(subtable0[resultLabel],return_counts=True)
            clValue1,countright = np.unique(subtable1[resultLabel],return_counts=True)
            if(len(countleft)<=1):
                pass
            else:
                root.left=buildTree(subtable0) #Calling the function recursively
            if(len(countright)<=1):
                pass
            else:
                root.right=buildTree(subtable1)


        return root

    # print(df)
    root=buildTree(df)
#     print(root.value)

#     print(root.right.value)
#     print(root.right.left)
#     print(root.left.value)
#     print(root.left.left.value)
#     print(root.left.right.value)
#     print(root.positive,root.negative)

    def testing(row1,root):
    #     if root==None:
    #         return 
        #print(root.value)
        if root.left==None and root.right==None:
            if root.positive > root.negative:
                y1.append(1)
            else:
                y1.append(0)
            return
    #     for i in row1.keys():
    #         if i==root.value:
        i=root.value
        if i in row1:
            
            if row1[i]==0:
        #         print("000")
                if root.left==None:
                    if root.positive > root.negative:
                        y1.append(1)
                        return
                    else:
                        y1.append(0)
                        return
                else:
                    testing(row1,root.left)
            else:
        #         print("111")
                if root.right==None:
                    if root.positive > root.negative:
                        y1.append(1)
                        return
                    else:
                        y1.append(0)
                        return 
                else:
                    testing(row1,root.right)
        else:
            if root.left==None:
                if root.positive > root.negative:
                    y1.append(1)
                    return
                else:
                    y1.append(0)
                    return
            else:
                testing(row1,root.left)



    #X_test
#     print(splitpoint)

    def testingData(df):
        for index,row in df.iterrows():
            testing(row,root)

    y1=[] 
    for col in numeric:
        value=splitpoint[col]
        for index,row1 in testXEntropy.iterrows():
                if float(row1[col])<value:
                    testXEntropy.at[index,col]=0
                else:
                    testXEntropy.at[index,col]=1
#     X_test
    testingData(testXEntropy)  
#     print(len(y1))


#     print(confusion_matrix(y_test, y1))  
#     print(accuracy_score(y_test,y1))
#     print(classification_report(y_test, y1)) 
    return y1

In [206]:
def CR(OrignalXCR,OrignalYCR,testXCR,testYCR):
    def findParentCR(df):
        if df.empty==True:
            return
        classCol=df.keys()[-1]
        #yeh sirf header return karega
        resultValues=df[classCol].unique()
        #all ouput values in output column
        gini=math.inf
        for value in resultValues:
            fraction=df[classCol].value_counts()[value]/(len(df[classCol])+eps)
            gini=min(gini,fraction)
        return gini
    def findRelativeCR(df,attribute):
        if df.empty==True:
            return
        classCol= df.keys()[-1] 
        #last col assumed as result 
        #getting target class 
        resultValues=df[classCol].unique() 
        attributeNames=df[attribute].unique()
        gini2 = 0
        for attr in attributeNames:
            gini = math.inf
            for value in resultValues:
                #temp&hot with yes
                num = len(df[attribute][df[attribute]==attr][df[classCol]==value])
                #total hot
                den = len(df[attribute][df[attribute]==attr])
                fraction = num/(den+eps)
                gini =min(fraction,gini)
            #entropy attribute/total*(entropy attribute_values)
            fraction2 = den/len(df)  
            gini2 += fraction2*gini
        return abs(gini2)  
    def get_subtable(df, node,value):
        return df[df[node] == value].reset_index(drop=True)
    def get_subtableSmaller(df, node,value):
        if df.empty==True:
            return
        return df[df[node] < value].reset_index(drop=True)

    def get_subtableGreater(df, node,value):
        if df.empty==True:
            return
        return df[df[node] >= value].reset_index(drop=True)

    def bestAttributeCR(df):
        if df.empty==True:
            return
        infoGain=[]
        for key in df.keys()[:-1]:
            infoGain.append(findParentCR(df)-findRelativeCR(df,key))
        return df.keys()[:-1][np.argmax(infoGain)]



    df=pd.concat([orignalXCR,orignalYCR],axis=1)
    print(df)
    splitpoint={}
    numeric=['last_evaluation','satisfaction_level','average_montly_hours','time_spend_company','number_project']
    for col in numeric:
        df.sort_values(col,inplace=True)
        prev=None;
        value=0;
        max1=0;
#         print(col)
        for index,row in df.iterrows():
            if prev is not None:
                if prev['left'] != row['left']:
                    mid=(float(prev[col])+float(row[col]))/2
                    subtableLeft=get_subtableSmaller(df,col,mid)
                    subtableRight=get_subtableGreater(df,col,mid)
                    fraction=len(subtableLeft)/len(df)
                    fraction1=len(subtableRight)/len(df)
                    Gini1=findParentCR(df)
    #                 print(subtableLeft)
    #                 a=input()
                    firstSubtable=0
                    secondSubtable=0
                    if subtableLeft.empty:
                        firstSubtable=0
                    else:
                        firstSubtable=fraction*findParentCR(subtableLeft)
    #                 print(subtableRight)
                    if subtableRight.empty:
                        secondSubtable=0
                    else:
                        secondSubtable=fraction1*findParentCR(subtableRight)

                    Gini2=firstSubtable+secondSubtable
                    ig1=abs(Gini1)-abs(Gini2)

                    if ig1>max1:
                        max1=ig1
                        value=mid
            prev=row
        splitpoint[col]=value 
        for index,row1 in orignalXCR.iterrows():
            if float(row1[col])<value:
                orignalXCR.at[index,col]=0
            else:
                orignalXCR.at[index,col]=1


    # print(X_train)
    # print(value)
    # print(max1)
    df=pd.concat([orignalXCR,orignalYCR],axis=1)
    #df

    class node:
        def __init__(self,value,left=None,right=None):
            self.value=value
            self.left=left
            self.right=right
            self.positive=0
            self.negative=0

    def buildTreeCR(df):
        if len(df.columns)==1:
            return

        resultLabel= df.keys()[-1] 
        clValue,counts = np.unique(df[resultLabel],return_counts=True)
        attribute_name=bestAttributeCR(df)
    #     attValue = np.unique(df[attribute_name])
        root=node(attribute_name)

        if len(counts)<=1:
            if clValue[0]==0:
                root.negative=counts[0]
            else:
                root.positive=counts[0]
        else:
            root.positive=counts[1]
            root.negative=counts[0]


    #         for value in attValue:
    #             print(type(value))

            subtable0 = get_subtable(df,attribute_name,0)
            subtable0 =subtable0.drop([attribute_name],axis=1)
            subtable1 = get_subtable(df,attribute_name,1)
            subtable1 =subtable1.drop([attribute_name],axis=1)
            clValue,countleft = np.unique(subtable0[resultLabel],return_counts=True)
            clValue1,countright = np.unique(subtable1[resultLabel],return_counts=True)
            if(len(countleft)<=1):
                pass
            else:
                root.left=buildTreeCR(subtable0) #Calling the function recursively
            if(len(countright)<=1):
                pass
            else:
                root.right=buildTreeCR(subtable1)


        return root

    # print(df)
    root=buildTreeCR(df)
#     print(root.value)

#     print(root.right.value)
#     print(root.right.left)
#     print(root.left.value)
#     print(root.left.left.value)
#     print(root.left.right.value)
#     print(root.positive,root.negative)

    def testing(row1,root):
    #     if root==None:
    #         return 
        #print(root.value)
        if root.left==None and root.right==None:
            if root.positive > root.negative:
                y1.append(1)
            else:
                y1.append(0)
            return
    #     for i in row1.keys():
    #         if i==root.value:
        i=root.value
        if i in row1:
            
            if row1[i]==0:
        #         print("000")
                if root.left==None:
                    if root.positive > root.negative:
                        y1.append(1)
                        return
                    else:
                        y1.append(0)
                        return
                else:
                    testing(row1,root.left)
            else:
        #         print("111")
                if root.right==None:
                    if root.positive > root.negative:
                        y1.append(1)
                        return
                    else:
                        y1.append(0)
                        return 
                else:
                    testing(row1,root.right)
        else:
            if root.left==None:
                if root.positive > root.negative:
                    y1.append(1)
                    return
                else:
                    y1.append(0)
                    return
            else:
                testing(row1,root.left)


    #X_test
#     print(splitpoint)

    def testingData(df):
        for index,row in df.iterrows():
            testing(row,root)

    y1=[] 
    for col in numeric:
        value=splitpoint[col]
        for index,row1 in testXCR.iterrows():
                if float(row1[col])<value:
                    testXCR.at[index,col]=0
                else:
                    testXCR.at[index,col]=1
#     testXGini
    testingData(testXCR)  
#     print(len(y1))
    return y1

In [207]:
y1=Entropy(orignalXEntropy,orignalYEntropy,testXEntropy,testYEntropy)
print(confusion_matrix(y_test, y1))  
print(accuracy_score(y_test,y1))
print(classification_report(y_test, y1)) 

       satisfaction_level  last_evaluation  number_project  \
1590                 0.09             0.96               6   
672                  0.96             0.93               5   
7014                 0.57             0.90               3   
7366                 0.45             0.49               2   
4647                 0.70             0.95               4   
248                  0.96             0.59               2   
5197                 0.39             0.55               2   
518                  0.53             0.97               4   
6454                 0.96             0.72               3   
10453                0.63             0.87               5   
11215                0.11             0.92               6   
7088                 0.30             0.67               3   
5632                 0.71             0.53               4   
4578                 0.85             0.84               5   
8351                 0.68             0.81               3   
2003    

[[1670   59]
 [ 224  295]]
0.8741103202846975
              precision    recall  f1-score   support

           0       0.88      0.97      0.92      1729
           1       0.83      0.57      0.68       519

   micro avg       0.87      0.87      0.87      2248
   macro avg       0.86      0.77      0.80      2248
weighted avg       0.87      0.87      0.87      2248



In [208]:
y1=Gini(orignalXGini,orignalYGini,testXGini,testYGini)
print(confusion_matrix(y_test, y1))  
print(accuracy_score(y_test,y1))
print(classification_report(y_test, y1)) 

       satisfaction_level  last_evaluation  number_project  \
1590                 0.09             0.96               6   
672                  0.96             0.93               5   
7014                 0.57             0.90               3   
7366                 0.45             0.49               2   
4647                 0.70             0.95               4   
248                  0.96             0.59               2   
5197                 0.39             0.55               2   
518                  0.53             0.97               4   
6454                 0.96             0.72               3   
10453                0.63             0.87               5   
11215                0.11             0.92               6   
7088                 0.30             0.67               3   
5632                 0.71             0.53               4   
4578                 0.85             0.84               5   
8351                 0.68             0.81               3   
2003    

[[1694   35]
 [ 202  317]]
0.8945729537366548
              precision    recall  f1-score   support

           0       0.89      0.98      0.93      1729
           1       0.90      0.61      0.73       519

   micro avg       0.89      0.89      0.89      2248
   macro avg       0.90      0.80      0.83      2248
weighted avg       0.90      0.89      0.89      2248



In [209]:
y1=CR(orignalXCR,orignalYCR,testXCR,testYCR)
print(confusion_matrix(y_test, y1))  
print(accuracy_score(y_test,y1))
print(classification_report(y_test, y1)) 

       satisfaction_level  last_evaluation  number_project  \
1590                 0.09             0.96               6   
672                  0.96             0.93               5   
7014                 0.57             0.90               3   
7366                 0.45             0.49               2   
4647                 0.70             0.95               4   
248                  0.96             0.59               2   
5197                 0.39             0.55               2   
518                  0.53             0.97               4   
6454                 0.96             0.72               3   
10453                0.63             0.87               5   
11215                0.11             0.92               6   
7088                 0.30             0.67               3   
5632                 0.71             0.53               4   
4578                 0.85             0.84               5   
8351                 0.68             0.81               3   
2003    

[[1683   46]
 [ 206  313]]
0.8879003558718861
              precision    recall  f1-score   support

           0       0.89      0.97      0.93      1729
           1       0.87      0.60      0.71       519

   micro avg       0.89      0.89      0.89      2248
   macro avg       0.88      0.79      0.82      2248
weighted avg       0.89      0.89      0.88      2248



In [210]:
classifier = DecisionTreeClassifier()
classifier.fit(orignalX, orignalY)  
y_pred = classifier.predict(testX)  
print(confusion_matrix(testY, y_pred))  
print(classification_report(testY, y_pred)) 
print(accuracy_score(testY,y_pred))

[[1682   47]
 [  23  496]]
              precision    recall  f1-score   support

           0       0.99      0.97      0.98      1729
           1       0.91      0.96      0.93       519

   micro avg       0.97      0.97      0.97      2248
   macro avg       0.95      0.96      0.96      2248
weighted avg       0.97      0.97      0.97      2248

0.9688612099644128
